# How to use
Run all and enter data when prompted

In [1]:
from torch.cuda import graph

# Get documents
documents = []

for i in range(int(input("Enter the number of documents for the AI to understand"))):
    documents.append(input("Copy the document here").replace("\n\n", ""))

documents

["Quiz bowl (quizbowl,[1] scholars' bowl, scholastic bowl, academic bowl, academic team, academic challenge, etc.) is a family of quiz-based competitions that test players on a wide variety of academic subjects. Standardized quiz bowl formats are played by primary school, middle school, high school, and university students throughout North America, Asia, Europe, Australia, and Africa.[2]Quiz bowl competitions are typically played with a lockout buzzer system[1] between at least two teams, usually consisting of four players each. A moderator reads questions to the players, who try to score points for their team by buzzing first and responding with the correct answer.",
 'Most forms of modern quiz bowl are modeled after game shows.[4] College Bowl, which was created by Don Reid as a USO activity for U.S. service men during World War II, was an influential early quiz bowl program.[5][6] Also known as "The College Quiz Bowl," it started on radio in 1953 and then aired on national televisio

In [35]:
question = input("Enter your question: ")
graph_gen = None

In [3]:
from GraphGen.graph_gen_integration import GraphGen
# Parse documents
triples = []
chunks = []

if graph_gen is None:
    graph_gen = GraphGen([], keyword_reward=1.3)


for idx, document in enumerate(documents):
    print(f"==================== Process document {idx+1}/{len(documents)} ====================")
    graph_gen.reset_model(triples, idx)
    triples += graph_gen.generate(document)
    chunks.append(graph_gen.split_chunks(document))
    
print("ACTUALLY DONE!")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.8: Fast Gemma2 patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA GeForce RTX 2060. Max memory: 6.0 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unsloth 2024.8 patched 26 layers with 26 QKV layers, 26 O layers and 26 MLP layers.


==================== Process document 1/2 ====================
PROCESSING (1/3)
Processing "Quiz bowl (quizbowl,[1] scholars' bowl, scholastic...nd Africa."
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Extract the most confident information in the sentence below as much as possible, and express the relationships in RDF Triples that complement the existing RDF triples. Do not use information from common sense.
### Existing RDF triples:
None
### Input:
Quiz bowl (quizbowl,[1] scholars' bowl, scholastic bowl, academic bowl, academic team, academic challenge, etc.) is a family of quiz-based competitions that test players on a wide variety of academic subjects. Standardized quiz bowl formats are played by primary school, middle school, high school, and university students throughout North America, Asia, Europe, Australia, and Africa.
### Response:
<unused0>
---
D

In [4]:
triples

[Triple(subject='Academic_Bowl', predicate='country', object='United_States', id=0, document_id=0),
 Triple(subject='Academic_Bowl', predicate='region', object='"Nationwide"', id=0, document_id=0),
 Triple(subject='Academic_Bowl', predicate='alternativeName', object='"Scholars\' Bowl"', id=0, document_id=0),
 Triple(subject='Academic_Bowl', predicate='sport', object='Quiz_bowl', id=0, document_id=0),
 Triple(subject='Quiz_bowl', predicate='alternativeName', object='"Scholars\' Bowl", "Academic Bowl", "Academic Team", "Academic Challenge", "Scholastic Bowl", "Primary School Quiz Bowl", "Middle School Quiz Bowl", "High School Quiz Bowl", "University Quiz Bowl"', id=0, document_id=0),
 Triple(subject='Academic_Bowl', predicate='location', object='"Nationwide"', id=1, document_id=0),
 Triple(subject='Academic_Bowl', predicate='numberOfPlayers', object='"Unlimited"', id=1, document_id=0),
 Triple(subject='Academic_Bowl', predicate='numberOfRounds', object='"Unlimited"', id=1, document_id=0)

In [5]:
from pyvis.network import Network
from GraphGen.graph_gen_integration import Triple
from GraphGen import graph_gen_integration

net = Network(bgcolor="#222222", font_color="white", notebook=True, directed=True)
net2 = Network(bgcolor="#222222", font_color="white", notebook=False, directed=True)


# Parse rdf_strings

def add_triples(rdf: Triple, _color: str):
    net.add_node(rdf.subject, color=_color)
    net.add_node(rdf.object, color=_color)
    # if not any(edge['from'] == rdf.subject and edge['to'] == rdf.object and edge['title'] == rdf.predicate for edge in net.edges): # should be deprecated later
    net.add_edge(rdf.subject, rdf.object, title=rdf.predicate, color=_color)
    
    net2.add_node(rdf.subject, color=_color)
    net2.add_node(rdf.object, color=_color)
    # if not any(edge['from'] == rdf.subject and edge['to'] == rdf.object and edge['title'] == rdf.predicate for edge in net.edges): # should be deprecated later
    net2.add_edge(rdf.subject, rdf.object, title=rdf.predicate, color=_color)


import random

r = lambda: random.randint(0, 255)
net.toggle_physics(True)

colors = {}

for idx, triple in enumerate(triples):
    if triple.document_id in colors:
        document = colors[triple.document_id]
        if triple.id in document:
            color = document[triple.id]
        else:
            color = '#%02X%02X%02X' % (r(), r(), r())
            colors[triple.document_id][triple.id] = color
    else:
        colors[triple.document_id] = {} 
        color = '#%02X%02X%02X' % (r(), r(), r())
        colors[triple.document_id][triple.id] = color

    add_triples(triple, color)
    # net.show(f"{idx}.html", notebook=False)
net.show("graph.html", notebook=True)
# net2.show("graph2.html", notebook=False) # For visualization outside notebook

graph.html


In [55]:
# Workaround for now. Backtracking to source
def get_chunk(hex_code):
    possible_docs = []
    for _document in colors:
        for _chunk in colors[_document]:
            if colors[_document][_chunk] == hex_code:
                possible_docs.append([_document, _chunk])
                
    if len(possible_docs) == 0:
        return None
    
    # print(possible_docs)
    
    docs = []
    
    for doc in possible_docs:
       docs.append(graph_gen.split_chunks(documents[doc[0]])[doc[1]])
    
    return docs

In [9]:
del graph_gen

In [37]:
import torch, gc
# graph_gen.free_model()
# del graph_gen
gc.collect()
torch.cuda.empty_cache()

from QueryGen.query_gen_integration import QueryGen
query_gen = QueryGen(triples)

[Triple(subject='Academic_Bowl', predicate='country', object='United_States', id=0, document_id=0), Triple(subject='Academic_Bowl', predicate='region', object='"Nationwide"', id=0, document_id=0), Triple(subject='Academic_Bowl', predicate='alternativeName', object='"Scholars\' Bowl"', id=0, document_id=0), Triple(subject='Academic_Bowl', predicate='sport', object='Quiz_bowl', id=0, document_id=0), Triple(subject='Quiz_bowl', predicate='alternativeName', object='"Scholars\' Bowl", "Academic Bowl", "Academic Team", "Academic Challenge", "Scholastic Bowl", "Primary School Quiz Bowl", "Middle School Quiz Bowl", "High School Quiz Bowl", "University Quiz Bowl"', id=0, document_id=0), Triple(subject='Academic_Bowl', predicate='location', object='"Nationwide"', id=1, document_id=0), Triple(subject='Academic_Bowl', predicate='numberOfPlayers', object='"Unlimited"', id=1, document_id=0), Triple(subject='Academic_Bowl', predicate='numberOfRounds', object='"Unlimited"', id=1, document_id=0), Tripl

In [38]:
# question = input("Enter your question: ")
query = query_gen.generate(question)
query

'SELECT DISTINCT ?uri WHERE {?uri <type> <QuizBowl>  . }'

In [39]:
from rdflib import Graph, Literal, RDF, URIRef
g = Graph()
for triple in triples:
    g.add((URIRef(triple.subject), URIRef(triple.predicate), URIRef(triple.object)))

"Nationwide" does not look like a valid URI, trying to serialize this will break.
"Scholars' Bowl" does not look like a valid URI, trying to serialize this will break.
"Scholars' Bowl", "Academic Bowl", "Academic Team", "Academic Challenge", "Scholastic Bowl", "Primary School Quiz Bowl", "Middle School Quiz Bowl", "High School Quiz Bowl", "University Quiz Bowl" does not look like a valid URI, trying to serialize this will break.
"Nationwide" does not look like a valid URI, trying to serialize this will break.
"Unlimited" does not look like a valid URI, trying to serialize this will break.
"Unlimited" does not look like a valid URI, trying to serialize this will break.
"Single-elimination tournament" does not look like a valid URI, trying to serialize this will break.
"4 per team" does not look like a valid URI, trying to serialize this will break.
"Buzzer" does not look like a valid URI, trying to serialize this will break.
"1953-09-28" does not look like a valid URI, trying to seriali

In [40]:
# Apply the query to the graph and iterate through results
r_r = []
for r in g.query(query):
    print(r[0])
    r_r.append(str(r[0]))
r_r

[]

In [45]:
query_gen.free_model()
del query_gen
gc.collect()
torch.cuda.empty_cache()

NameError: name 'query_gen' is not defined

In [42]:
from AnswerGen.answer_gen_integration import AnswerGen
answer_gen = AnswerGen()

answer = answer_gen.generate(question, str(r_r))
answer

==((====))==  Unsloth 2024.8: Fast Gemma2 patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA GeForce RTX 2060. Max memory: 6.0 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


'The quizbowl is the sport of the quiz bowl.<eos>'

In [46]:
answer_gen.free_model()
del answer_gen
gc.collect()
torch.cuda.empty_cache()

In [43]:
print(answer.replace("<eos>", ""))

The quizbowl is the sport of the quiz bowl.


In [ ]:
graph_gen = GraphGen([], keyword_reward=1.1)
graph_gen.reset_model(triples, idx)

In [56]:
# CHUNK LOOKUP
hex_code = input("Enter hex code for chunk lookup")
chunks = get_chunk(hex_code)
if chunks is None:
    print("No related chunk found. Try again")
if len(chunks) == 1:
    print("CHUNK: ")
    print(chunks[0])
if len(chunks) > 1:
    print(f"{len(chunks)} chunks somehow got assigned the same color. Find them below")
    for chunk in chunks:
        print(chunk)

CHUNK: 
[8] The It's Academic televised student quiz show program has been run for high school teams in the Washington, D.C., metropolitan area since 1961 and is recognized by the Guinness Book of World Records as the longest-running quiz program in television history. [9] It's Academic has been spun off in many other U.S. media markets and has inspired many other televised high school competitions. [9]


In [50]:
colors

{0: {0: '#2B6E1E', 1: '#A29881', 2: '#74686B'},
 1: {0: '#355423', 1: '#752F2E', 2: '#A19E54', 3: '#B0681C', 4: '#B5BCC9'}}